<a href="https://colab.research.google.com/github/Anudeep003/MainProject/blob/main/trained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
# Cell 0
!pip install -q scikit-learn xgboost joblib streamlit pyngrok
# cloudflared doesn't need pip; will download binary later


In [106]:
# Cell 1
import os, pprint
print("Current working directory:", os.getcwd())
print("\nFiles at /content (first 200 entries):")
p = os.listdir("/content")
p.sort()
pprint.pprint(p[:200])
print("\nMake sure these CSVs are uploaded to /content:")
print(" - dataset.csv")
print(" - Symptom-severity.csv")
print(" - symptom_Description.csv (optional)")
print(" - symptom_precaution.csv (optional)")


Current working directory: /content

Files at /content (first 200 entries):
['.config',
 'Symptom-severity.csv',
 '__pycache__',
 'app.py',
 'artifacts',
 'artifacts.zip',
 'cloudflared',
 'dataset.csv',
 'main.py',
 'sample_data',
 'symptom_Description.csv',
 'symptom_precaution.csv',
 'uvicorn.log']

Make sure these CSVs are uploaded to /content:
 - dataset.csv
 - Symptom-severity.csv
 - symptom_Description.csv (optional)
 - symptom_precaution.csv (optional)


In [107]:
# Cell 2
import os, shutil, pandas as pd
if os.path.exists("dataset.csv"):
    if not os.path.exists("dataset_orig_backup.csv"):
        shutil.copy("dataset.csv", "dataset_orig_backup.csv")
        print("Backed up dataset.csv -> dataset_orig_backup.csv")
    else:
        print("Backup already exists: dataset_orig_backup.csv")
else:
    raise FileNotFoundError("dataset.csv not found in /content. Upload before continuing.")

df = pd.read_csv("dataset.csv")
print("Original dataset shape:", df.shape)
display(df.head(6))
symptom_cols = [c for c in df.columns if 'Symptom' in c or c.lower().startswith('symptom_')]
print("Detected symptom columns:", symptom_cols)
empty_sym = df[symptom_cols].fillna('').apply(lambda row: all([str(x).strip()=='' for x in row]), axis=1).sum()
print(empty_sym, "rows with no symptoms (will be removed).")
print("Exact duplicate rows count:", df.duplicated().sum())
print("Disease value counts (top 20):")
print(df['Disease'].value_counts().head(20))


Backed up dataset.csv -> dataset_orig_backup.csv
Original dataset shape: (4920, 18)


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Detected symptom columns: ['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5', 'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10', 'Symptom_11', 'Symptom_12', 'Symptom_13', 'Symptom_14', 'Symptom_15', 'Symptom_16', 'Symptom_17']
0 rows with no symptoms (will be removed).
Exact duplicate rows count: 4616
Disease value counts (top 20):
Disease
Fungal infection                120
Allergy                         120
GERD                            120
Chronic cholestasis             120
Drug Reaction                   120
Peptic ulcer diseae             120
AIDS                            120
Diabetes                        120
Gastroenteritis                 120
Bronchial Asthma                120
Hypertension                    120
Migraine                        120
Cervical spondylosis            120
Paralysis (brain hemorrhage)    120
Jaundice                        120
Malaria                         120
Chicken pox                     120
Dengue           

In [108]:
# Cell 3
import re, pandas as pd
df = pd.read_csv("dataset.csv")
symptom_cols = [c for c in df.columns if 'Symptom' in c or c.lower().startswith('symptom_')]

def normalize_sym(s):
    if pd.isna(s): return ''
    s = str(s).strip().lower()
    s = re.sub(r"[^\w\s\-\/\(\)]", " ", s)
    s = s.replace("/", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

for c in symptom_cols:
    df[c] = df[c].fillna('').astype(str).apply(normalize_sym)

df['Disease'] = df['Disease'].astype(str).str.strip()

# remove rows with no symptoms
df['__no_symptoms'] = df[symptom_cols].apply(lambda row: all([ (str(x).strip()=='' or str(x).strip().lower() in ['nan','none']) for x in row]), axis=1)
before = df.shape[0]
df = df[~df['__no_symptoms']].copy()
after_no_empty = df.shape[0]
print(f"Removed {before-after_no_empty} rows with no symptoms.")

# drop exact duplicates
before = df.shape[0]
df = df.drop_duplicates()
after_dups = df.shape[0]
print(f"Dropped {before-after_dups} exact duplicate rows.")

df = df.drop(columns=['__no_symptoms'])
df.to_csv("dataset_clean_step1.csv", index=False)
print("Saved dataset_clean_step1.csv")
display(df.head(6))


Removed 0 rows with no symptoms.
Dropped 4616 exact duplicate rows.
Saved dataset_clean_step1.csv


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,,,,,,,,,,,,,
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,,,,,,,,,,,,,,
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,,,,,,,,,,,,,,
3,Fungal infection,itching,skin_rash,dischromic _patches,,,,,,,,,,,,,,
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,,,,,,,,,,,,,,
10,Allergy,continuous_sneezing,shivering,chills,watering_from_eyes,,,,,,,,,,,,,


In [109]:
# Cell 4
import pandas as pd
df = pd.read_csv("dataset_clean_step1.csv")
symptom_cols = [c for c in df.columns if 'Symptom' in c or c.lower().startswith('symptom_')]

def symptom_set_key(row):
    vals = []
    for c in symptom_cols:
        v = str(row[c]).strip().lower()
        if v and v not in ['nan','none','']:
            vals.append(v)
    vals_sorted = sorted(list(set(vals)))
    return "||".join(vals_sorted)

df['__symset'] = df.apply(symptom_set_key, axis=1)
counts = df.groupby(['Disease'])['__symset'].nunique().sort_values(ascending=False)
print("Unique symptom-sets per Disease (top 20):")
print(counts.head(20))

before = df.shape[0]
df_coll = df.drop_duplicates(subset=['Disease','__symset']).copy()
after = df_coll.shape[0]
print(f"Collapsed identical symptom-sets per Disease: {before} -> {after} rows")

df_coll = df_coll.drop(columns=['__symset'])
df_coll.to_csv("dataset_clean_step2.csv", index=False)
print("Saved dataset_clean_step2.csv")
display(df_coll.head(6))


Unique symptom-sets per Disease (top 20):
Disease
Chicken pox            10
Hepatitis D            10
Migraine               10
Dengue                 10
Common Cold             9
Diabetes                9
Hepatitis B             9
Hepatitis E             9
Hyperthyroidism         9
Pneumonia               9
Tuberculosis            9
hepatitis A             9
Typhoid                 9
Hypoglycemia            9
Jaundice                9
Hypothyroidism          8
Alcoholic hepatitis     8
Varicose veins          8
Malaria                 8
Chronic cholestasis     8
Name: __symset, dtype: int64
Collapsed identical symptom-sets per Disease: 304 -> 304 rows
Saved dataset_clean_step2.csv


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Allergy,continuous_sneezing,shivering,chills,watering_from_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
# Cell 5
import pandas as pd
df = pd.read_csv("dataset_clean_step2.csv")
MAX_PER_DISEASE = 200   # set to None to disable capping

if MAX_PER_DISEASE is not None:
    frames = []
    for disease, group in df.groupby('Disease'):
        if len(group) > MAX_PER_DISEASE:
            group = group.sample(n=MAX_PER_DISEASE, random_state=42)
        frames.append(group)
    df_capped = pd.concat(frames, ignore_index=True)
else:
    df_capped = df.copy()

df_capped.to_csv("dataset_clean_step3.csv", index=False)
print("Saved dataset_clean_step3.csv; shape:", df_capped.shape)
print("Sample disease counts:")
print(df_capped['Disease'].value_counts().head(20))


Saved dataset_clean_step3.csv; shape: (304, 18)
Sample disease counts:
Disease
Chicken pox            10
Hepatitis D            10
Migraine               10
Dengue                 10
Common Cold             9
Diabetes                9
Hepatitis B             9
Hepatitis E             9
Hyperthyroidism         9
Pneumonia               9
Tuberculosis            9
hepatitis A             9
Typhoid                 9
Hypoglycemia            9
Jaundice                9
Hypothyroidism          8
Alcoholic hepatitis     8
Varicose veins          8
Malaria                 8
Chronic cholestasis     8
Name: count, dtype: int64


In [111]:
# Cell 6
import shutil, os
# choose final
for candidate in ["dataset_clean_step3.csv","dataset_clean_step2.csv","dataset_clean_step1.csv"]:
    if os.path.exists(candidate):
        final = candidate
        break
else:
    raise FileNotFoundError("No cleaned dataset file found. Run cleaning steps first.")

shutil.copy(final, "dataset.csv")
print("Overwrote dataset.csv with cleaned file:", final)
df_final = pd.read_csv("dataset.csv")
print("Final dataset shape:", df_final.shape)
print("Top Disease counts:")
print(df_final['Disease'].value_counts().head(30))
df_final.head(6)


Overwrote dataset.csv with cleaned file: dataset_clean_step3.csv
Final dataset shape: (304, 18)
Top Disease counts:
Disease
Chicken pox                                10
Hepatitis D                                10
Migraine                                   10
Dengue                                     10
Common Cold                                 9
Diabetes                                    9
Hepatitis B                                 9
Hepatitis E                                 9
Hyperthyroidism                             9
Pneumonia                                   9
Tuberculosis                                9
hepatitis A                                 9
Typhoid                                     9
Hypoglycemia                                9
Jaundice                                    9
Hypothyroidism                              8
Alcoholic hepatitis                         8
Varicose veins                              8
Malaria                                     8
Ch

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,loss_of_balance,unsteadiness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(vertigo) Paroymsal Positional Vertigo,headache,nausea,spinning_movements,loss_of_balance,unsteadiness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,(vertigo) Paroymsal Positional Vertigo,vomiting,nausea,spinning_movements,loss_of_balance,unsteadiness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,spinning_movements,loss_of_balance,unsteadiness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,loss_of_balance,unsteadiness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,unsteadiness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
# Cell 7
import pandas as pd, numpy as np
dataset = pd.read_csv("dataset.csv")
symptom_severity = pd.read_csv("Symptom-severity.csv")

# detect symptom columns
symptom_cols = [c for c in dataset.columns if 'Symptom' in c or c.lower().startswith('symptom_')]

# normalize dataset symptom entries
for c in symptom_cols:
    dataset[c] = dataset[c].astype(str).replace('nan','', regex=False).fillna('').astype(str)

all_symptoms = set()
for c in symptom_cols:
    for v in dataset[c].unique():
        v = str(v).strip().lower()
        if v and v not in ['nan','none','']:
            all_symptoms.add(v)
all_symptoms = sorted(list(all_symptoms))
print("Number of unique symptoms:", len(all_symptoms))

# create one-hot columns efficiently
sym_cols_new = ["sym_" + s.replace(" ", "_") for s in all_symptoms]
onehot_df = pd.DataFrame(0, index=dataset.index, columns=sym_cols_new)

for c in symptom_cols:
    vals = dataset[c].astype(str).str.strip().str.lower()
    for i, val in vals.items():
        if val and val not in ['nan','none','']:
            colname = "sym_" + val.replace(" ", "_")
            if colname in onehot_df.columns:
                onehot_df.at[i, colname] = 1

dataset = pd.concat([dataset.reset_index(drop=True), onehot_df.reset_index(drop=True)], axis=1)
feature_cols = sym_cols_new[:]   # symptom one-hot columns

# severity map
symptom_severity.columns = [c.strip() for c in symptom_severity.columns]
symptom_severity['Symptom_norm'] = symptom_severity[symptom_severity.columns[0]].astype(str).str.strip().str.lower()
severity_map = dict(zip(symptom_severity['Symptom_norm'], symptom_severity['weight'].astype(float)))

def compute_severity_row(row):
    total = 0.0
    for s in all_symptoms:
        col = "sym_" + s.replace(" ", "_")
        if col in row and row[col] == 1:
            total += severity_map.get(s, 0.0)
    return total

dataset['severity_score'] = dataset.apply(compute_severity_row, axis=1)
print("Severity score summary:")
display(dataset['severity_score'].describe())

# Save intermediate for inspection
dataset.to_csv("dataset_ready_for_training.csv", index=False)
print("Saved dataset_ready_for_training.csv")


Number of unique symptoms: 131
Severity score summary:


,severity_score
count,304.000000
mean,31.707237
std,16.575355
min,4.000000
25%,19.000000
50%,28.000000
75%,42.000000
max,77.000000


Saved dataset_ready_for_training.csv


In [113]:
# Cell 8
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import joblib

feature_cols_full = feature_cols + ['severity_score']
X = dataset[feature_cols_full].astype(float)
y_raw = dataset['Disease'].astype(str).str.strip()
le = LabelEncoder()
y = le.fit_transform(y_raw)
print("Num disease classes:", len(le.classes_))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

# save label encoding for inspection
joblib.dump(le, "label_encoder_temp.pkl")
print("Saved temporary label_encoder_temp.pkl")


Num disease classes: 41
Train shape: (243, 132) Test shape: (61, 132)
Saved temporary label_encoder_temp.pkl


In [114]:
# Cell 9
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib, numpy as np

clf = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced', n_jobs=-1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification report (short):")
print(classification_report(y_test, y_pred, target_names=le.classes_))


Accuracy: 1.0
Classification report (short):
                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00         1
                                   AIDS       1.00      1.00      1.00         1
                                   Acne       1.00      1.00      1.00         1
                    Alcoholic hepatitis       1.00      1.00      1.00         2
                                Allergy       1.00      1.00      1.00         1
                              Arthritis       1.00      1.00      1.00         1
                       Bronchial Asthma       1.00      1.00      1.00         1
                   Cervical spondylosis       1.00      1.00      1.00         1
                            Chicken pox       1.00      1.00      1.00         2
                    Chronic cholestasis       1.00      1.00      1.00         2
                            Common Cold       1.00      1.00   

In [115]:
# Cell 10
import numpy as np, os, joblib
train_sev = X_train['severity_score']
low_thr = float(np.percentile(train_sev, 33))
high_thr = float(np.percentile(train_sev, 66))
print("Severity thresholds: low =", low_thr, "high =", high_thr)

os.makedirs("artifacts", exist_ok=True)
joblib.dump(clf, "artifacts/predictcare_model_rf.pkl")
joblib.dump(le, "artifacts/label_encoder.pkl")
joblib.dump(feature_cols_full, "artifacts/feature_cols.pkl")
joblib.dump(severity_map, "artifacts/severity_map.pkl")
joblib.dump({"low": low_thr, "high": high_thr}, "artifacts/severity_thresholds.pkl")
print("Saved artifacts to ./artifacts/")
!ls -l artifacts


Severity thresholds: low = 21.86 high = 38.0
Saved artifacts to ./artifacts/
total 11704
-rw-r--r-- 1 root root     2848 Nov 14 21:25 feature_cols.pkl
-rw-r--r-- 1 root root     1132 Nov 14 21:25 label_encoder.pkl
-rw-r--r-- 1 root root 11968401 Nov 14 21:25 predictcare_model_rf.pkl
-rw-r--r-- 1 root root     3504 Nov 14 21:25 severity_map.pkl
-rw-r--r-- 1 root root       47 Nov 14 21:25 severity_thresholds.pkl


In [116]:
# Cell 11
import joblib, pandas as pd, numpy as np
model = joblib.load("artifacts/predictcare_model_rf.pkl")
le = joblib.load("artifacts/label_encoder.pkl")
saved_feature_cols = joblib.load("artifacts/feature_cols.pkl")
severity_map = joblib.load("artifacts/severity_map.pkl")
thr = joblib.load("artifacts/severity_thresholds.pkl")

def infer_from_symptoms_local(symptom_list):
    s_norm = [s.strip().lower() for s in symptom_list if s and isinstance(s, str)]
    fv = {col: 0 for col in saved_feature_cols}
    for s in s_norm:
        col = "sym_" + s.replace(" ", "_")
        if col in fv:
            fv[col] = 1
    sev = float(sum(severity_map.get(s, 0.0) for s in s_norm))
    fv["severity_score"] = sev
    X_new = pd.DataFrame([fv], columns=saved_feature_cols).astype(float)
    if hasattr(model, "predict_proba"):
        pred_proba = model.predict_proba(X_new)[0]
    else:
        preds = model.predict(X_new)
        pred_proba = np.zeros(len(le.classes_)); pred_proba[preds[0]] = 1.0
    top_idx = pred_proba.argsort()[::-1][:5]
    return [{"disease": le.inverse_transform([i])[0], "probability": float(pred_proba[i])} for i in top_idx], sev

print(infer_from_symptoms_local(["fever","cough","headache"]))


([{'disease': 'Paralysis (brain hemorrhage)', 'probability': 0.22}, {'disease': 'Fungal infection', 'probability': 0.16}, {'disease': 'Acne', 'probability': 0.14}, {'disease': 'GERD', 'probability': 0.08}, {'disease': 'Bronchial Asthma', 'probability': 0.07}], 7.0)


In [117]:
# Cell 12
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib, os, traceback

st.set_page_config(page_title="PredictCare", layout="centered")

ARTIFACT_PATH = "artifacts"
model = joblib.load(os.path.join(ARTIFACT_PATH, "predictcare_model_rf.pkl"))
le = joblib.load(os.path.join(ARTIFACT_PATH, "label_encoder.pkl"))
saved_feature_cols = joblib.load(os.path.join(ARTIFACT_PATH, "feature_cols.pkl"))
severity_map = joblib.load(os.path.join(ARTIFACT_PATH, "severity_map.pkl"))
thr = joblib.load(os.path.join(ARTIFACT_PATH, "severity_thresholds.pkl"))

if "severity_score" not in saved_feature_cols:
    saved_feature_cols = list(saved_feature_cols) + ["severity_score"]

model_feature_names = None
if hasattr(model, "feature_names_in_"):
    model_feature_names = list(model.feature_names_in_)

final_cols = model_feature_names if model_feature_names is not None else saved_feature_cols

sym_cols = [c for c in final_cols if c.startswith("sym_")]
symptoms = [c[len("sym_"):].replace("_"," ") for c in sym_cols]
symptoms_sorted = sorted(symptoms)

def infer_from_symptoms(symptoms_list):
    s_norm = [s.strip().lower() for s in symptoms_list if s and isinstance(s, str)]
    fv = {c: 0 for c in final_cols}
    for s in s_norm:
        col = "sym_" + s.replace(" ", "_")
        if col in fv:
            fv[col] = 1
    sev = float(sum(severity_map.get(s, 0.0) for s in s_norm))
    if "severity_score" in fv:
        fv["severity_score"] = sev
    X_new = pd.DataFrame([fv], columns=final_cols).astype(float)
    try:
        if hasattr(model, "predict_proba"):
            pred_proba = model.predict_proba(X_new)[0]
        else:
            preds = model.predict(X_new)
            pred_proba = np.zeros(len(le.classes_)); pred_proba[preds[0]] = 1.0
    except Exception as e:
        tb = traceback.format_exc()
        raise RuntimeError(f"Prediction failed. Diagnostics: final_cols length={len(final_cols)}, X_new.shape={X_new.shape}. Original exception: {e}\\nTraceback:\\n{tb}")
    top_idx = pred_proba.argsort()[::-1][:5]
    results = [{"disease": le.inverse_transform([i])[0], "probability": float(pred_proba[i])} for i in top_idx]
    if thr.get("low") is None or thr.get("high") is None:
        risk = "unknown"
    else:
        if sev < thr["low"]:
            risk = "low"
        elif sev < thr["high"]:
            risk = "medium"
        else:
            risk = "high"
    return {"predictions": results, "severity_score": sev, "risk": risk}

desc_df = None
prec_df = None
for candidate in ["symptom_Description.csv","symptom_precaution.csv"]:
    if os.path.exists(candidate):
        try:
            df = pd.read_csv(candidate)
            if "Description" in df.columns:
                desc_df = df
            if any("precaution" in c.lower() for c in df.columns):
                prec_df = df
        except Exception:
            pass

def get_description_and_precautions(disease_name):
    desc = "Description not found."
    precautions = []
    if desc_df is not None:
        r = desc_df[desc_df['Disease'].str.strip().str.lower() == disease_name.strip().lower()]
        if len(r):
            desc = r.iloc[0].get("Description", desc)
    if prec_df is not None:
        r = prec_df[prec_df['Disease'].str.strip().str.lower() == disease_name.strip().lower()]
        if len(r):
            row = r.iloc[0]
            for c in r.columns:
                if 'precaution' in c.lower():
                    val = row.get(c)
                    if pd.notnull(val) and str(val).strip():
                        precautions.append(str(val).strip())
    return desc, precautions

st.title("PredictCare — Symptom → Disease Predictor")
st.markdown("Select symptoms (or add custom ones). App will use exact feature names/order from training.")

selected = st.multiselect("Select known symptoms:", options=symptoms_sorted, default=[])
extra = st.text_input("Add other symptom(s) (comma-separated):", value="")

if st.button("Predict"):
    extras = [s.strip() for s in extra.split(",") if s.strip()]
    user_syms = selected + extras
    if not user_syms:
        st.warning("Choose at least one symptom.")
    else:
        try:
            out = infer_from_symptoms(user_syms)
        except Exception as e:
            st.error("Prediction error - see details below.")
            st.text(str(e))
        else:
            st.subheader("Top predictions")
            for p in out["predictions"]:
                st.write(f"- **{p['disease']}** — probability: {p['probability']:.3f}")
            st.write(f"**Severity score:** {out['severity_score']:.1f}")
            st.write(f"**Risk level:** {out['risk'].upper()}")
            top = out["predictions"][0]["disease"]
            desc, precs = get_description_and_precautions(top)
            st.subheader(f"About {top}")
            st.write(desc)
            if precs:
                st.subheader("Precautions")
                for i,p in enumerate(precs, start=1):
                    st.write(f"{i}. {p}")


Overwriting app.py


In [118]:
# Cell 13
get_ipython().system_raw("streamlit run app.py &")
import time
time.sleep(2)
print("Streamlit processes:")
!ps aux | grep streamlit | grep -v grep || true


Streamlit processes:
root       62595 54.0  0.3  63348 53116 ?        R    21:26   0:01 /usr/bin/python3 /usr/local/bin/streamlit run app.py


In [119]:
# Cell 14
!curl -L -o cloudflared "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64"
!chmod +x cloudflared
!./cloudflared --version


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 39.3M  100 39.3M    0     0  26.1M      0  0:00:01  0:00:01 --:--:-- 59.0M
cloudflared version 2025.11.1 (built 2025-11-07-16:59 UTC)


In [120]:
# Cell 15
!./cloudflared tunnel --url http://localhost:8501
# Keep this cell running and look for a line: "Started tunnel at https://<random>.trycloudflare.com"


2025-11-14T21:26:43Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-14T21:26:43Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-14T21:26:47Z INF +--------------------------------------------------------------------------------------------+
2025-11-14T21:26:47Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-14T21:26:47Z INF |  https://munich-monkey-email-ata.trycloudflare.com    